In [51]:
import os
import pandas as pd
import numpy as np


# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy
app = Flask(__name__)

In [52]:
# Step 1 - create our database engine
engine = create_engine("sqlite:///bellybutton.sqlite") 

# Step 2 - create connection object for binding purpose
conn = engine.connect()

# Step 3: create a session object that will be used for querying purposes at table level
session = Session(bind=conn)

# Step 4: use inspect method to check all or a specific table
inspector = inspect(conn)
tables = inspector.get_table_names()
for table in tables:
    print("------- Schema details for table: " + table + "----")
    columns = inspector.get_columns(table)
    for c in columns:
        print(c['name'], c["type"])
    print("--------------------------------------------")


------- Schema details for table: sample_metadata----
sample INTEGER
EVENT TEXT
ETHNICITY TEXT
GENDER TEXT
AGE REAL
WFREQ REAL
BBTYPE TEXT
LOCATION TEXT
COUNTRY012 TEXT
ZIP012 TEXT
COUNTRY1319 TEXT
ZIP1319 TEXT
DOG TEXT
CAT TEXT
IMPSURFACE013 REAL
NPP013 REAL
MMAXTEMP013 REAL
PFC013 REAL
IMPSURFACE1319 TEXT
NPP1319 REAL
MMAXTEMP1319 REAL
PFC1319 REAL
--------------------------------------------
------- Schema details for table: samples----
otu_id INTEGER
otu_label TEXT
940 INTEGER
941 INTEGER
943 INTEGER
944 INTEGER
945 INTEGER
946 INTEGER
947 INTEGER
948 INTEGER
949 INTEGER
950 INTEGER
952 INTEGER
953 INTEGER
954 INTEGER
955 INTEGER
956 INTEGER
958 INTEGER
959 INTEGER
960 INTEGER
961 INTEGER
962 INTEGER
963 INTEGER
964 INTEGER
966 INTEGER
967 INTEGER
968 INTEGER
969 INTEGER
970 INTEGER
971 INTEGER
972 INTEGER
973 INTEGER
974 INTEGER
975 INTEGER
978 INTEGER
1233 INTEGER
1234 INTEGER
1235 INTEGER
1236 INTEGER
1237 INTEGER
1238 INTEGER
1242 INTEGER
1243 INTEGER
1246 INTEGER
1253 INTEGER


In [53]:
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///bellybutton.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table
Samples_Metadata = Base.classes.sample_metadata
Samples = Base.classes.samples

C:\Users\Janie228\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [54]:
def names():
    """Return a list of sample names."""

    # Use Pandas to perform the sql query
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Return a list of the column names (sample names)
    return jsonify(list(df.columns)[2:])

In [55]:
stmt = db.session.query(Samples).statement
df = pd.read_sql_query(stmt, db.session.bind)
t_df = df.columns[2:]
t_df

Index(['940', '941', '943', '944', '945', '946', '947', '948', '949', '950',
       ...
       '1562', '1563', '1564', '1572', '1573', '1574', '1576', '1577', '1581',
       '1601'],
      dtype='object', length=153)

In [56]:
def sample_metadata(sample):
    """Return the MetaData for a given sample."""
    sel = [
        Samples_Metadata.sample,
        Samples_Metadata.ETHNICITY,
        Samples_Metadata.GENDER,
        Samples_Metadata.AGE,
        Samples_Metadata.LOCATION,
        Samples_Metadata.BBTYPE,
        Samples_Metadata.WFREQ,
    ]
    
    results = db.session.query(*sel).filter(Samples_Metadata.sample == sample).all()

    # Create a dictionary entry for each row of metadata information
    sample_metadata = {}
    for result in results:
        sample_metadata["sample"] = result[0]
        sample_metadata["ETHNICITY"] = result[1]
        sample_metadata["GENDER"] = result[2]
        sample_metadata["AGE"] = result[3]
        sample_metadata["LOCATION"] = result[4]
        sample_metadata["BBTYPE"] = result[5]
        sample_metadata["WFREQ"] = result[6]

    print(sample_metadata)
    return jsonify(sample_metadata)


In [62]:
stmt = db.session.query("940").statement
df = pd.read_sql_query(stmt, db.session.bind)
df

,940
0,940


In [57]:
def samples(sample):
    """Return `otu_ids`, `otu_labels`,and `sample_values`."""
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Filter the data based on the sample number and
    # only keep rows with values above 1
    sample_data = df.loc[df[sample] > 1, ["otu_id", "otu_label", sample]]
    # Format the data to send as json
    data = {
        "otu_ids": sample_data.otu_id.values.tolist(),
        "sample_values": sample_data[sample].values.tolist(),
        "otu_labels": sample_data.otu_label.tolist(),
    }
    return jsonify(data)

In [58]:
test = sample_metadata(940)
test

{'sample': 940, 'ETHNICITY': 'Caucasian', 'GENDER': 'F', 'AGE': 24.0, 'LOCATION': 'Beaufort/NC', 'BBTYPE': 'I', 'WFREQ': 2.0}


RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.

In [63]:
test2 = samples(940)
test2

KeyError: 940